# 1. 메타 학습

## 웜 스타트란?
- 탐색 알고리즘 대부분은 임의로 설정한 초깃값부터 탐색을 시작하며, 초깂값에 따라 탐색 시간과 성능이 좌우됨
- 웜 스타트는 적절한 초깃값을 설정하여 탐색을 시작하는 것으로, 탐색 시간을 줄이고 좋은 성능의 해를 찾는 것을 도와줌

## 메타 모델을 활용한 웜 스타트
- 메타 학습이란 데이터에 관한 데이터인 메타 데이터(meta data)로 모델을 학습하는 것을 의미하며, 이렇게 학습한 모델을 메타 모델이라고 합니다.
- 메타 모델은 데이터와 모델 정보를 입력하면 예상되는 성능을 반환합니다.

## 자주 사용하는 메타 특징
- 머신러닝 자동화 시스템에서 주로 사용하는 메타 특징은 다음과 같습니다.

자주 사용하는 메타 특징

|특징|설명|특징|설명|
|---|---|---|---|
|샘플 개수|데이터에 포함된 샘플 개수|수치형 특징 개수|데이터에 포함된 수치형 특징 개수|
|특징 개수|데이터에 포함된 특징 개수|범주형 특징 개수|데이터에 포함된 범주형 특징 개수|
|클래스 대비 특징 비율|특징 개수 / 클래스 개수|수치형 특징 비율|수치형 특징 개수 / 특징 개수|
|클래스 비율 분포|예) 각 클래스 비율의 최댓값, 각 클래스 비율의 최솟값|범주형 특징 비율|범주형 특징 개수 / 특징 개수|
|연속형 라벨의 분포|예) 라벨의 범위, 라벨의 평균, 라벨의 표준편차|특징 간 상관관계 분포|특징 간 상관관계의 최댓값, 최솟값 등|
|샘플 대비 특징 비율|특징 개수 / 샘플 개수| | |

- 클래스 비율 분포와 특징 간 상관관계 분포를 통해 분포를 요약하는 통계량(예: 평균, 최댓값 등)을 메타 특징으로 사용함을 알 수 있음
- 예를 들어, 두 측징 X1과 X2 간 상관관계가 아니라 가능한 두 특징 간 상관관계의 통계량을 사용함
- 그 이유는 데이터마다 특징 개수, 클래스 개수 등이 다르므로, 각 특징 및 클래스에 대한 값을 메타 특징으로 사용하면 데이터마다 메타 특징 개수가 달라지기 때문

어느 데이터에 대해서나 동일하게 추출할 수 있고 모델 성능에 영향을 끼치는 메타 특징을 정의하는 것이 중요함

## 메타 모델의 종류
- 어떠한 지도 학습 모델도 메타 모델로 사용할 수 있지만, 적은 데이터로도 학습이 잘되고 특징의 중요도를 반영할 수 있는 모델이 좋습니다.
- 메타 모델 요구 사항
 - 일바적으로 메타 모델은 데이터 확보가 어려우므로 적은 데이터로 학습하더라도 잘 작동해야 함
 - 메타 특징, 모델, 하이퍼 파라미터의 중요도를 조절할 수 있어야 함

# 2. 실습 1- 데이터 준비

## 메타 데이터 준비
- autoMPG8.csv 데이터로 신경망의 하이퍼 파라미터를 튜닝할 때 초깃값을 설정하기 위한 메타 학습을 해보겠습니다.

메타 데이터 준비

In [1]:
import pandas as pd
data_path = '../data/regression/'
meta_file_list = [  'abalone.csv',
                    'autoMPG6.csv',
                    'baseball.csv',
                    'friedman.csv',
                    'stock.csv',
                    'wankara.csv']

meta_data_list = []
for file in meta_file_list:
    df = pd.read_csv(data_path + '/' + file)
    X = df.drop('y', axis = 1)
    y = df['y']
    meta_data_list.append((X, y))

- 라인 11~15: meta_file_list에 정의된 파일 이름을 순회하면서 데이터를 불러오고 특징과 라벨로 분리한 뒤 튜플 형태로 meta_data_list에 추가합니다. 즉, meta_data_list[i][0]은 i번째 데이터의 특징 벡터를, meta_data_list[i][1]은 i번째 데이터의 라벨을 나타냅니다.

실제로는 머신러닝 자동화 시스템이 구축돼 새로운 데이터가 끊임없이 들어올 때만 혹은 들어올 것이라 예상될 때만 메타 학습을 사용하며, 한 데이터에 대해서만 메타 모델을 적용하는 것은 매우 비효율적임

## 메타 특징 추출
- 메타 특징을 추출하는 함수를 작성하겠습니다

메타 특징 추출 함수

In [2]:
def extract_meta_features(X, y):
    num_samples, num_features = X.shape
    label_max = y.max()
    label_min = y.min()
    label_mean = y.mean()
    label_std = y.std()
    corr_mean = X.corr().abs().values.mean()
    corr_max = X.corr().abs().values.max()
    corr_min = X.corr().abs().values.min()

    meta_features = [   num_samples,
                        num_features,
                        label_max,
                        label_min,
                        label_mean,
                        label_std,
                        corr_mean,
                        corr_max,
                        corr_min]
    
    return meta_features

- 라인 2: shape 메서드를 이용해 X의 행 개수와 열 개수를 각각 num_samples와 num_features에 저장합니다.
- 라인 3~6: y의 최댓값, 최솟값, 평균, 표준편차를 각각 label_max, label_min, label_mean, label_std에 저장합니다.
- 라인 7~9: 특징 간 상관관계의 절댓값의 평균, 최댓값, 최솟값을 각각 corr_mean, corr_max, corr_min에 저장합니다. 이때 데이터프레임의 집계 메서드를 사용하면 열별 통계량을 반환하므로 ndarray로 변환 후에 집계 메서드를 사욯했습니다.

각 데이터로부터 메타 특징을 추출하겠습니다.

메타 특징 추출

In [3]:
meta_data = []
for X, y in meta_data_list:
    meta_features = extract_meta_features(X, y)
    meta_data.append(meta_features)

meta_col_names = [  'num_samples',
                    'num_features',
                    'label_max',
                    'label_min',
                    'label_mean',
                    'label_std',
                    'corr_mean',
                    'corr_max',
                    'corr_min']

meta_data = pd.DataFrame(meta_data, columns = meta_col_names)
meta_data['data_name'] = meta_file_list

- 라인 2~4: meta_data_list에 속한 X와 y로부터 메타 특징을 추출하고 그 값을 meta_data에 추가합니다.
- 라인 17: 각 데이터 이름을 나타내는 data_name 칼럼을 추가합니다. 이 칼럼은 하이퍼 파라미터 튜닝 결과 데이터와 병합하겠습니다.

## 메타 데이터 생성
- 하이퍼 파라미터를 샘플링하는 함수를 다음과 같이 작성하겠습니다.

하이퍼 파라미터 샘플러

In [4]:
import numpy as np
def hyperparameter_sampling():
    h1 = np.random.randint(5, 15)
    h2, h3, h4 = np.random.randint(0, 10, 3)
    if h2 == 0:
        h3, h4 = 0, 0
    elif h3 == 0:
        h4 =0
    max_iter = np.random.choice([100, 200, 1000, 2000])
    random_state = np.random.choice([2020, 2021, 2022])
    return h1, h2, h3, h4, max_iter, random_state

- 라인 3: 첫 번째 은닉층에 포함될 노드 수 h1을 5와 15 사이에서 임의로 샘플링합니다. 첫 번째 은닉층에 포함될 녿는 0보다 커야 하므로 따로 샘플링했습니다.
- 라인 4: 두 번째, 세 번째, 네 번째 은닉층에 포함될 노드 수 h2, h3, h4를 0부터 10 사이에서 임의로 샘플링합니다.
- 라인 5~6: 두 번째 은닉층에 포함될 노드 수가 0이면 세 번째와 네 번째에 포함될 노드 수도 0으로 설정합니다.
- 라인 7~8: 세 번째 은닉층에 포함될 노드 수가 0이면 네 번째에 포함될 노드 수도 0으로 설정합니다.
- 라인 9: 최대 이터레이션 횟수 max_iter를 100, 20, 1000, 2000 중 하나로 설정합니다.

데이터마다 1,000 번씩 하이퍼 파라미터를 샘플링하고 그 때의 성능을 experiment_data에 저장하겠습니다.

In [5]:
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPRegressor as MLP
import warnings
from tqdm import tqdm
warnings.filterwarnings('ignore')
experiment_data = []

- 라인 5: 코드 실행 과정을 출력하기 위해 tqdm 함수를 불러왔습니다.
- 라인 6: 불필요한 경고가 뜨는 것을 방지했습니다.

데이터마다 1,000번씩 하이퍼 파라미터를 샘플링하고 그때의 성능을 experiment_data에 저장하겠습니다.

In [6]:
for i in range(len(meta_data_list)):
    data_name = meta_data_list[i]
    X, y = meta_data_list[i]
    print(data_name)
    for j in tqdm(range(1000)):
        h1, h2, h3, h4, max_iter, random_state = hyperparameter_sampling()
        if h2 == 0:
            layers = (h1, )
        elif h3 == 0:
            layers = (h1, h2)
        elif h4 == 0:
            layers = (h1, h2, h3)
        else:
            layers = (h1, h2, h3, h4)
        
        model = MLP(    hidden_layer_sizes = layers,
                        max_iter = max_iter,
                        random_state = random_state )
        score_list = - cross_val_score(model, X, y, cv = 5,
                                    scoring = 'neg_mean_absolute_error')
        score = score_list.mean()
        record = [data_name, h1, h2, h3, h4, max_iter, random_state, score]
        experiment_data.append(record)

hyper_param_cols = ['h1', 'h2', 'h3', 'h4', 'max_iter', 'random_state']
experiment_data = pd.DataFrame(experiment_data,
                                columns = ['data_name'] + hyper_param_cols + ['score'])

(      x1     x2     x3     x4      x5      x6      x7      x8
0      3  0.400  0.305  0.100  0.3415  0.1760  0.0625  0.0865
1      2  0.635  0.500  0.150  1.3760  0.6495  0.3610  0.3100
2      3  0.370  0.270  0.090  0.1855  0.0700  0.0425  0.0650
3      1  0.680  0.540  0.155  1.5340  0.6710  0.3790  0.3840
4      3  0.375  0.285  0.090  0.2545  0.1190  0.0595  0.0675
...   ..    ...    ...    ...     ...     ...     ...     ...
4171   1  0.535  0.405  0.140  0.7315  0.3360  0.1560  0.1900
4172   3  0.365  0.270  0.085  0.1970  0.0815  0.0325  0.0650
4173   3  0.555  0.430  0.140  0.7665  0.3410  0.1650  0.2300
4174   3  0.485  0.380  0.120  0.4725  0.2075  0.1075  0.1470
4175   2  0.550  0.450  0.145  0.7410  0.2950  0.1435  0.2665

[4176 rows x 8 columns], 0        7
1       10
2        7
3       10
4        6
        ..
4171     7
4172     6
4173     9
4174     6
4175    10
Name: y, Length: 4176, dtype: int64)


100%|██████████| 1000/1000 [41:32:35<00:00, 149.56s/it]      


(        x1   x2    x3    x4  x5
0     91.0   70  1955  20.5  71
1    232.0  100  2789  15.0  73
2    350.0  145  4055  12.0  76
3    318.0  140  4080  13.7  78
4    113.0   95  2372  15.0  70
..     ...  ...   ...   ...  ..
387  225.0  105  3121  16.5  73
388  454.0  220  4354   9.0  70
389   85.0   65  1975  19.4  81
390  350.0  165  3693  11.5  70
391  307.0  130  3504  12.0  70

[392 rows x 5 columns], 0      26.0
1      18.0
2      13.0
3      17.5
4      24.0
       ... 
387    18.0
388    14.0
389    37.0
390    15.0
391    18.0
Name: y, Length: 392, dtype: float64)


100%|██████████| 1000/1000 [1:15:21<00:00,  4.52s/it]


(        x1     x2   x3   x4  x5  x6  x7   x8  x9  x10  x11  x12  x13  x14  \
0    0.271  0.328   74  161  22   6  12   58  49  133   23   17    1    1   
1    0.264  0.318   24   48   7   0   1   22  15   18    0    7    0    0   
2    0.251  0.338  101  141  35   3  32  105  71  104   34    6    0    0   
3    0.224  0.274   28   94  21   1   1   44  27   54    2    7    1    1   
4    0.206  0.262   14   51  18   1   1   28  17   26    0    3    1    1   
..     ...    ...  ...  ...  ..  ..  ..  ...  ..  ...  ...  ...  ...  ...   
332  0.269  0.303   32   58  12   1  13   33  11   51    1    2    0    0   
333  0.288  0.366   24   47   7   0   7   24  18   23    2    3    0    0   
334  0.208  0.265   12   35  11   1   0   15  14   30    2    6    0    0   
335  0.290  0.349   59  141  30   2  16   64  42  102   14    6    1    0   
336  0.225  0.312   37   64  12   0  17   48  36   80    1    0    1    0   

     x15  x16  
0      0    0  
1      0    0  
2      1    0  
3      0  

...

100%|██████████| 1000/1000 [3:53:22<00:00, 14.00s/it] 


(         x1      x2      x3      x4      x5      x6      x7      x8      x9
0    17.219  50.500  18.750  43.000  60.875  26.375  67.750  19.000  48.750
1    17.891  51.375  19.625  44.000  62.000  26.125  68.125  19.125  48.750
2    18.438  50.875  19.875  43.875  61.875  27.250  68.500  18.250  49.000
3    18.672  51.500  20.000  44.000  62.625  27.875  69.375  18.375  49.625
4    17.438  49.000  20.000  41.375  59.750  25.875  63.250  16.500  47.500
..      ...     ...     ...     ...     ...     ...     ...     ...     ...
945  50.375  46.250  19.375  52.250  61.875  23.500  78.625  26.625  41.875
946  50.750  46.375  19.625  50.875  64.625  23.250  77.625  26.500  40.750
947  50.625  46.625  19.625  50.875  64.625  23.250  75.000  26.250  41.250
948  50.125  47.000  19.875  50.750  62.750  22.875  74.500  25.250  40.625
949  49.000  47.000  19.500  49.500  60.875  22.750  75.625  25.500  40.500

[950 rows x 9 columns], 0      34.875
1      35.625
2      36.375
3      36.250
4     

100%|██████████| 1000/1000 [2:40:21<00:00,  9.62s/it] 


(        x1    x2    x3    x4     x5     x6   x7     x8     x9
0     46.4  29.1  26.2  0.00  29.79  26.53  8.4   4.72  14.90
1     87.8  57.9  51.5  0.05  29.72  26.69  8.6   8.98  25.30
2     86.7  57.2  51.1  0.00  29.95  26.82  8.7   9.44  12.70
3     75.2  42.1  41.3  0.00  29.79  26.66  8.7   3.80  33.20
4     42.8  30.6  34.9  0.00  30.16  26.89  6.5   2.07   9.21
...    ...   ...   ...   ...    ...    ...  ...    ...    ...
1603  79.9  52.2  48.1  0.00  29.91  26.83  8.8   5.64  13.31
1604  86.0  46.8  43.9  0.40  29.83  26.70  8.5   5.87  12.70
1605  45.3  30.6  32.0  0.00  30.06  26.76  8.7   9.32  16.10
1606  57.2  35.6  33.4  0.00  29.83  26.64  8.8  15.10  27.50
1607  45.3  28.4  26.6  0.00  30.00  26.71  9.0   3.11  11.30

[1608 rows x 9 columns], 0       37.9
1       72.3
2       73.5
3       58.5
4       41.1
        ... 
1603    64.9
1604    69.2
1605    40.2
1606    47.2
1607    36.0
Name: y, Length: 1608, dtype: float64)


100%|██████████| 1000/1000 [3:18:15<00:00, 11.90s/it] 


- 라인 8~10: range(meta_data_list)를 순회하는 방식으로 meta_file_list와 meta_data_list를 순회합니다.
- 라인 11~12: 코드 실행 기간이 기므로, data_name을 출력하고 tqdm을 사용해 진행률을 출력합니다.
- 라인 13: hyperparameter_sampling 함수를 사용해 h1, h2, h3, h4, max_iter, random_state를 샘플링합니다.
- 라인 14~21: h2, h3, h4의 값에 따라 layers를 설정합니다. 예를 들어, h2가 0 이라면 layers를 (h1,)으로, h3가 0이라면 layers를 (h1, h2)로 설정합니다.
- 라인 23~30: 샘플링한 하이퍼 파라미터를 갖는 신경망을 k-겹 교차 검증을 통해 평가합니다. 또, 그 결과를 record에 추가하고 record를 experiment_data에 추가합니다.
- 라인 32~34: experiment_data를 데이터프레임으로 변환합니다.

meta_data에 experiment_data를 부착하여 meta_data를 완성하겠습니다.

In [1]:
meta_data = pd.merge(   meta_data,
                        experiment_data,
                        on = 'data_name')
display(meta_data.head())

NameError: name 'pd' is not defined

# 3. 실습 2- 메타 모델 학습 및 활용

## 메테 모델 학습
- 메타 모델로 k-최근접 이웃 모델을 사용하겠습니다. 하이퍼 파라미터 튜닝은 따로 하지 않았습니다.

메타 모델 학습

In [ ]:
from sklearn.neighbors import KNeighborsRegressor as KNN
from sklearn.preprocessing import MinMaxScaler
meta_X = meta_data.drop(['data_name', 'score'], axis = 1)
meta_X['random_state_2020'] = (meta_X['random_state'] == 2020).astype(int)
meta_X['random_state_2021'] = (meta_X['random_state'] == 2021).astype(int)
meta_X = meta_X.drop(['random_state'], axis = 1)
meta_y = meta_data['score']
scaler = MinMaxScaler().fit(meta_X)
meta_X = scaler.transform(meta_X)
meta_model = KNN().fit(meta_X, meta_y)

- 라인 3: meta_data에서 특징으로 활용할 수 없는 data_name과 score를 제거합니다.
- 라인 4~6: random_state는 범주형 변수이므로 범주화합니다. 이 변수만 범주형 변수이므로 직접 더미화를 했습니다.
- 라인 8~9: meta_X를 0과 1 사이로 스케일링합니다.

## 메타 모델 활용
- 새로 입력된 데이터 autoMPG8.csv에 대해 메타 모델을 사용해서 초깃값을 설정하겠습니다.

메타 모델 활용

In [ ]:
df = pd.read_csv(data_path + '/autoMPG8.csv')
X = df.drop('y', axis = 1)
y = df['y']
meta_features = extract_meta_features(X, y)
sample_list = []
for _ in range(100):
    h1, h2, h3, h4, max_iter, random_state = hyperparameter_sampling()
    if h2 == 0:
        layers = (h1, )
    elif h3 == 0:
        layers = (h1, h2)
    elif h4 == 0:
        layers = (h1, h2, h3)
    else:
        layers = (h1, h2, h3, h4)
    sample_list.append(meta_features + [h1, h2, h3, h4, max_iter, random_state])

sample_list = pd.DataFrame( sample_list,
                            columns = meta_col_names + hyper_param_cols)

sample_list['random_state_2020'] = (sample_list['random_state'] == 2020).astype(int)
sample_list['random_state_2021'] = (sample_list['random_state'] == 2021).astype(int)
sample_list = sample_list.drop(['random_state'], axis = 1)
y_pred = meta_model.predict(scaler.transform(sample_list))

In [ ]:
init_sample = sample_list.loc[np.argsort(y_pred)[:3]]
display(init_sample)

## 메타 모델 검증
- sample_list에 속한 값의 라벨인 y_actual을 계산하여 비교해보겠습니다.

메타 모델 검증

In [ ]:
y_actual = []
for sample in sample_list:
    h1, h2, h3, h4, max_iter, random_state = sample[-6:]
    if h2 == 0:
        layers = (h1, )
    elif h3 == 0:
        layers = (h1, h2)
    elif h4 == 0:
        layers = (h1, h2, h3)
    else:
        layers = (h1, h2, h3, h4)
    
    model = MLP(    hidden_layer_sizes = layers,
                    max_iter = max_iter,
                    random_state = random_state)
    
    score_list = - cross_val_score( model, X, y, cv = 5,
                                    scoring = 'neg_mean_absolute_error')
    
    score = score_list.mean()
    y_actual.append(score)

- 라인 2~3: sample_list에 있는 요소를 sample로 순회하면서 하이퍼 파라미터와 관련된 부분만 입력 받습니다. 왜냐하면 새로운 데이터의 메타 특징은 하나여서 샘플링할 때마다 바뀌지 않으므로 학습에 전혀 도움이 되지 않기 때문입니다.

y_actual과 y_pred 간의 스피어만 상관계수(Spearman correlation coefficient)를 계산해보겠습니다. 스피어만 상관계수는 두 변수의 순위 간 상관관계를 나타내는 지표로 두 변수의 스케일에 영향을 받지 않습니다.

메타 모델 검증

In [ ]:
from scipy.stats import spearmanr, rankdata
print(spearmanr(y_actual, y_pred))
print(rankdata(y_actual)[np.argsort(y_pred)[:3]])

- 라인 2: y_actual과 y_pred 간 스피엄나 상관계수를 계산합니다.
- 라인 3: 메타 모델이 MAE가 가장 작으리라 예측한 세 개이 해가 실제 평가 결과에서 몇 위를 차지하는지를 출력합니다.

- 예측값과 실제값 간 스피어만 상관계수가 0.6191로, 두 변수 간 양의 상관관계가 존재함.
- 또한, 예측 결과에서 1등 부터 3등 까지라고 판단한 세 개의 샘플은 실제로는 13등, 26등, 7등으로, 다소 아쉽기는 하지만 사용하기 어려운 수준은 아님